# Problem 5.2  

**Integrated Energy Grids**

**Problem 5.2**

**Assume we have one bus (Denmark) in which there is a gas power generator whose variable cost is 50 EUR/MWh and installed capacity is 50 MW, and a wind generator whose variable cost is zero and whose installed capacity is 30 MW. Using Python for Power System Analysis (PyPSA):**

**a) Calculate the optimal dispatch that minimizes the total system cost, the energy produced by each generator, and the electricity price assuming that power demand is 80 MW**

**b) Calculate the optimal dispatch that minimizes the total system cost, the energy produced by each generator, and the electricity price assuming that power demand is 20 MW**

_Note: This is a straightforward problem, whose objective is to showcase how to solve one-node economic dispatch problems in PyPSA. It is recommended to follow the [PyPSA tutorial](https://martavp.github.io/integrated-energy-grids/intro-pypsa.html#) before trying this problem._

:::{note}
If you have not yet set up Python on your computer, you can execute this tutorial in your browser via [Google Colab](https://colab.research.google.com/). Click on the rocket in the top right corner and launch "Colab". If that doesn't work download the `.ipynb` file and import it in [Google Colab](https://colab.research.google.com/).

Then install the following packages by executing the following command in a Jupyter cell at the top of the notebook.

```sh
!pip install numpy pypsa
```
:::

In [1]:
import numpy as np
import pypsa

We start by creating the network object and adding the bus.

In [2]:
network = pypsa.Network()

In [3]:
network.add("Bus", 
            "bus Denmark") 
network.buses

,v_nom,type,x,y,carrier,unit,v_mag_pu_set,v_mag_pu_min,v_mag_pu_max,control,generator,sub_network
Bus,,,,,,,,,,,,
bus Denmark,1.0,,0.0,0.0,AC,,1.0,0.0,inf,PQ,,


We add the generators

In [4]:
network.add("Generator", 
            "gas Denmark", 
            bus="bus Denmark", 
            p_nom=50, 
            marginal_cost=50) #EUR/MWh_elec
    
network.add("Generator", 
            "wind Denmark", 
            bus="bus Denmark", 
            p_nom=30, 
            marginal_cost=0)
network.generators

,bus,control,type,p_nom,p_nom_mod,p_nom_extendable,p_nom_min,p_nom_max,p_min_pu,p_max_pu,...,min_up_time,min_down_time,up_time_before,down_time_before,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down,weight,p_nom_opt
Generator,,,,,,,,,,,,,,,,,,,,,
gas Denmark,bus Denmark,PQ,,50.0,0.0,False,0.0,inf,0.0,1.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0
wind Denmark,bus Denmark,PQ,,30.0,0.0,False,0.0,inf,0.0,1.0,...,0,0,1,0,NaN,NaN,1.0,1.0,1.0,0.0


We add the load

In [6]:
network.add("Load", 
            "load Demmark", 
            bus="bus Denmark", 
            p_set=80)
network.loads

,bus,carrier,type,p_set,q_set,sign,active
Load,,,,,,,
load Demmark,bus Denmark,,,80.0,0.0,-1.0,True


Calculate the optimal economic dispatch

In [7]:
network.optimize()

Index(['bus Denmark'], dtype='object', name='Bus')
Index(['bus Denmark'], dtype='object', name='Bus')
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io: Writing time: 0.02s
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 2 primals, 5 duals
Objective: 2.50e+03
Solver model: available
Solver message: optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper were not assigned to the network.


('ok', 'optimal')

Now we can show the optimal dispatch and the electricity price (marginal cost at the bus)

In [8]:
network.generators_t.p

Generator,gas Denmark,wind Denmark
snapshot,,
now,50.0,30.0


In [9]:
network.buses_t.marginal_price

Bus,bus Denmark
snapshot,
now,50.0


We update the load and calculate the economic dispatch again. 

In [10]:
network.add("Load", 
            "load Demmark", 
            bus="bus Denmark", 
            p_set=20, 
            overwrite=True)
network.optimize()

Index(['bus Denmark'], dtype='object', name='Bus')
Index(['bus Denmark'], dtype='object', name='Bus')
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io: Writing time: 0.02s
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 2 primals, 5 duals
Objective: 0.00e+00
Solver model: available
Solver message: optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper were not assigned to the network.


('ok', 'optimal')

In [11]:
network.generators_t.p

Generator,gas Denmark,wind Denmark
snapshot,,
now,-0.0,20.0


In [12]:
network.buses_t.marginal_price

Bus,bus Denmark
snapshot,
now,-0.0
